# Hypothesis testing problem

### **Problem 1**

#### ANOVA

Suppose that a study wants to check if there is a significant difference between the goal averages of soccer players depending on the position in which they play. In case there is a difference, you want to know which positions differ from the rest.

NOTE: You must replace the values <<<FIXME>>>.

**Exercise: Load data from "datos_laliga.csv". It contains a sample of randomly selected players.**

In [ ]:
import pandas as pd
my_data = pd.read_csv(<<<FIXME>>>)
my_data

In [ ]:
<<<FIXME>>>.describe()

**Exercise: Identify the number of groups and number of observations per group to determine if it is a balanced model. The mean and standard deviation of the group are also calculated.**

In [ ]:
pd.crosstab(my_data[<<<FIXME>>>],columns=["DC", "MO", "MP", "P"])
#DC: Delantero centro
#MO: Medio ofensivo
#MP: Media punta
#P: Puntero

**Exercise: Calculate the mean and standar deviation by position**

In [ ]:
my_data.groupby('<<<FIXME>>>')['average'].agg('<<<FIXME>>>')

**Exercise: Calculate the standard deviation by position**

In [ ]:

my_data.groupby('<<<FIXME>>>')['average'].agg('<<<FIXME>>>')

Since the number of observations per group is not constant, it is an unbalanced model. It is important to take this into account when checking the conditions of normality and homoscedasticity (s1 = s2 = s3 = s4). The most useful graphical representation before performing an ANOVA is the Box-Plot model.

**Exercise: Plot a boxplot for each position**

In [ ]:
# Boxplots
# This is another way and popular package used in R (ggplot)
from plotnine import *

(
    ggplot(my_data)  # What data to use
    + aes(x="<<<FIXME>>>", y="average")  # What variable to use
    + geom_boxplot()  # Geometric object to use for drawing
    + theme_bw()
)

#### Independence

The total sample size is <10% of the population of all players in the league. The groups (categorical variable) are independent of each other since a random sample of players from the entire league (not just from the same team) has been made.

Normal distribution of observations: The quantitative variable must be distributed in a normal way in each of the groups. The normality study can be done graphically (qqplot) or with a hypothesis test.

**Exercise: Make an analysis about normal distribution for each position**

In [ ]:
import numpy as np 
import pylab 
import matplotlib.pyplot as plt
import scipy.stats as stats

measurements = my_data.loc[my_data["<<<FIXME>>>"] == "<<<FIXME>>>","average"]
stats.probplot(measurements, dist="norm", plot=pylab)
plt.title("P")
plt.show()

measurements = my_data.loc[my_data["<<<FIXME>>>"] == "<<<FIXME>>>","average"]
stats.probplot(measurements, dist="norm", plot=pylab)
plt.title("MO")
plt.show()

measurements = my_data.loc[my_data["<<<FIXME>>>"] == "<<<FIXME>>>","average"]
stats.probplot(measurements, dist="norm", plot=pylab)
plt.title("DC")
plt.show()

measurements = my_data.loc[my_data["<<<FIXME>>>"] == "<<<FIXME>>>","average"]
stats.probplot(measurements, dist="norm", plot=pylab)
plt.title("MP")
plt.show()

**Exercise: make the boxplot for each position, what you can say about them?**

In [ ]:
# Using plotly
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Box(y=my_data.loc[my_data["<<<FIXME>>>"] == "<<<FIXME>>>","average"]))
fig.add_trace(go.Box(y=my_data.loc[my_data["<<<FIXME>>>"] == "<<<FIXME>>>","average"]))
fig.add_trace(go.Box(y=my_data.loc[my_data["<<<FIXME>>>"] == "<<<FIXME>>>","average"]))
fig.add_trace(go.Box(y=my_data.loc[my_data["<<<FIXME>>>"] == "<<<FIXME>>>","average"]))
fig.show()

**Exercise: Use the Kolmogorov-Smirnov test or with or without the Lilliefors correction in order to know the normality distribution.**

In [ ]:
# Beginner way to do it

from statsmodels.stats.diagnostic import lilliefors

my_df = pd.DataFrame(index=np.arange(len(np.unique(my_data["position"]))), columns=["position", "D_statistic", "p_value"])
my_df["position"] = np.unique(my_data["position"])

for position in my_df["position"]:
    my_data_subset = my_data.loc[my_data["position"] == position,:]
    D_statistic, p_value = lilliefors(my_data_subset.average)
    my_df.loc[my_df["position"]==position,["D_statistic", "p_value"]] = D_statistic, p_value
    
print(my_df)

In [ ]:
#Another way to do it (Highly recommendable)
my_data.groupby("<<<FIXME>>>")["average"].apply(<<<FIXME>>>)

The hypothesis tests do not show evidence of a lack of normality.

Constant variance between groups (homoscedasticity):

Given that there is a group (DC) that is at the limit to accept that it is distributed in a normal way, the Fisher and Bartlett tests are not recommended. Instead it is better to use a test based on the median Levene test or the Fligner-Killeen test.

**Exercise: use the fligner and levene functions from scipy.stats in order to know the homocedasticy**

In [ ]:
from scipy import stats

values_array = pd.DataFrame(my_data.groupby("<<<FIXME>>>")["average"]).to_numpy()

print(stats.fligner(values_array[0,1], values_array[1,1], values_array[2,1], values_array[3,1]))

print(stats.levene(values_array[0,1], values_array[1,1], values_array[2,1], values_array[3,1]))

# stats.fligner(values_array[:,1]) # It doesn't work, please analyse

There is no significant evidence of lack of homoscedasticity in either of the two tests.

The study of the conditions can be carried out after calculating the ANOVA, since if they are not fulfilled, it does not make much sense to continue. However, the most appropriate way to verify that the necessary conditions are satisfied is by studying the model residuals once the ANOVA has been generated.

**Exercise: make an ANOVA table and analyze the p-value using the packages statsmodels and the bioinfokit**

In [ ]:
# get ANOVA table as R like output
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Ordinary Least Squares (OLS) model
model = ols('<<<FIXME>>> ~ <<<FIXME>>>', data=my_data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

In [ ]:
# ANOVA with package bioinfokit
from bioinfokit.analys import stat
res = stat()
res.anova_stat(df=my_data, res_var='value', anova_model='<<<FIXME>>> ~ <<<FIXME>>>')
res.anova_summary
# output (ANOVA F and p value)

**Exercise: make a plot of the fitted values vs residuals. Make the plot of the Standardized Residuals. Make the histogram of the residuals.**

In [ ]:
# QQ-plot
import statsmodels.api as sm
import matplotlib.pyplot as plt


plt.scatter(res.anova_model_out.<<<FIXME>>>, res.anova_model_out.<<<FIXME>>>)
plt.xlabel("Fitted values")
plt.ylabel("Residuals")
plt.title("Residuals vs. Fitted")
plt.show()

# res.anova_std_residuals are standardized residuals obtained from ANOVA (check above)
sm.qqplot(res.<<<FIXME>>>, line='45')
plt.xlabel("Theoretical Quantiles")
plt.ylabel("Standardized Residuals")
plt.show()

# histogram
plt.hist(res.anova_model_out.<<<FIXME>>>, bins='auto', histtype='bar', ec='k') 
plt.xlabel("Residuals")
plt.ylabel('Frequency')
plt.show()

Given that the p-value is higher than 0.05, there is not enough evidence to consider that at least two means are different. The graphical representation of the residuals does not show lack of homoscedasticity (graph 1) and in the qqplot the residuals are distributed very close to the normal line (graph 2 and 3).

### **Problem 2**

A maternity hospital wants to see the effect of formula consumption on the average monthly weight gain (in gr) of babies. For this reason, she collected data from three different groups. The first group is exclusively breastfed children(receives only breast milk), the second group is children who are fed with only formula and the last group is both formula and breastfed children. These data are as below

```py
only_breast=[794.1, 716.9, 993. , 724.7, 760.9, 908.2, 659.3 , 690.8, 768.7, 717.3 , 630.7, 729.5, 714.1, 810.3, 583.5, 679.9, 865.1]

only_formula=[ 898.8, 881.2, 940.2, 966.2, 957.5, 1061.7, 1046.2, 980.4, 895.6, 919.7, 1074.1, 952.5, 796.3, 859.6, 871.1 , 1047.5, 919.1 , 1160.5, 996.9]

both=[976.4, 656.4, 861.2, 706.8, 718.5, 717.1, 759.8, 894.6, 867.6, 805.6, 765.4, 800.3, 789.9, 875.3, 740. , 799.4, 790.3, 795.2 , 823.6, 818.7, 926.8, 791.7, 948.3]
```

According to this information, conduct the hypothesis testing to check whether there is a difference between the average monthly gain of these three groups by using a 0.05 significance level. If there is a significant difference, perform further analysis to find what caused the difference. Before doing hypothesis testing, check the related assumptions. Comment on the results.

In [5]:
import numpy as np
from scipy import stats
import pandas as pd

In [1]:
def check_normality(data):
    test_stat_normality, p_value_normality=stats.shapiro(data)
    print("p value:%.4f" % p_value_normality)
    if p_value_normality <0.05:
        print("Reject null hypothesis >> The data is not normally distributed")
    else:
        print("Fail to reject null hypothesis >> The data is normally distributed")       

In [2]:
def check_variance_homogeneity(group1, group2):
    test_stat_var, p_value_var= stats.levene(group1,group2)
    print("p value:%.4f" % p_value_var)
    if p_value_var <0.05:
        print("Reject null hypothesis >> The variances of the samples are different.")
    else:
        print("Fail to reject null hypothesis >> The variances of the samples are same.")


### Solution

In [6]:
only_breast=np.array([794.1, 716.9, 993. , 724.7, 760.9, 908.2, 659.3 , 690.8, 768.7,
       717.3 , 630.7, 729.5, 714.1, 810.3, 583.5, 679.9, 865.1])

only_formula=np.array([ 898.8,  881.2,  940.2,  966.2,  957.5, 1061.7, 1046.2,  980.4,
        895.6,  919.7, 1074.1,  952.5,  796.3,  859.6,  871.1 , 1047.5,
        919.1 , 1160.5,  996.9])

both=np.array([976.4, 656.4, 861.2, 706.8, 718.5, 717.1, 759.8, 894.6, 867.6,
       805.6, 765.4, 800.3, 789.9, 875.3, 740. , 799.4, 790.3, 795.2 ,
       823.6, 818.7, 926.8, 791.7, 948.3])

H₀: The data is normally distributed.

H₁: The data is not normally distributed.

In [7]:
check_normality(only_breast)
check_normality(only_formula)
check_normality(both)

p value:0.4694
Fail to reject null hypothesis >> The data is normally distributed
p value:0.8879
Fail to reject null hypothesis >> The data is normally distributed
p value:0.7973
Fail to reject null hypothesis >> The data is normally distributed


H₀: The variances of the samples are the same.

H₁: The variances of the samples are different.

In [8]:
stat, pvalue_levene= stats.levene(only_breast,only_formula,both)

print("p value:%.4f" % pvalue_levene)
if pvalue_levene <0.05:
    print("Reject null hypothesis >> The variances of the samples are different.")
else:
    print("Fail to reject null hypothesis >> The variances of the samples are same.")

p value:0.7673
Fail to reject null hypothesis >> The variances of the samples are same.


H₀:  u1 = u2 = u3 or The mean of the samples is the same.

H₁: At least one of them is different.

In [9]:
F, p_value = stats.f_oneway(only_breast,only_formula,both)
print("p value:%.6f" % p_value)
if p_value <0.05:
    print("Reject null hypothesis")
else:
    print("Fail to reject null hypothesis")

p value:0.000000
Reject null hypothesis


At this significance level, it can be concluded that at least one of the groups has a different average monthly weight gain.

----> Pairwise T test for multiple comparisons of independent groups. May be used after a parametric ANOVA to do pairwise comparisons.